<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/7_add_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import ast
import joblib

In [ ]:
# RAAP = pd.read_csv("/.RAAP.csv")
# raap = pd.read_csv("raap.csv")

In [ ]:
full_ra_rap = pd.read_excel("full_ra_rap.xlsx")

In [ ]:
full_rap = full_ra_rap[full_ra_rap["סוג הליך"]=='רע"פ']
# בדיקה אם בעמודת "גוף המסמך" קיימת המילה "רע"פ"
full_rap = full_rap[full_rap["גוף המסמך"].str.contains('רע"פ', na=False)]

In [ ]:
full_rap

In [ ]:
full_rap.shape

In [ ]:
full_rap[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(full_rap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
full_rap["גוף המסמך"] = full_rap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(full_rap["גוף המסמך"].head())

file_path = 'full_rap_df_cleaned.csv'
full_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


In [ ]:
# בדיקה אם הערכים השתנו
changed_rows = full_rap["גוף המסמך"] != full_rap["גוף המסמך"].apply(clean_text)
print(f"מספר שורות ששונו: {changed_rows.sum()}")
print("שורות ששונו:")
print(full_rap.loc[changed_rows, "גוף המסמך"].head())


In [ ]:
full_rap

In [ ]:
# הגדרת פונקציה לעיצוב טקסט
def format_text(text, line_length=80):
    """
    פורמט טקסט כך שיהיה נוח לקריאה עם שורות שאורכן מוגבל.
    """
    import textwrap
    return "\n".join(textwrap.wrap(text, width=line_length))


In [ ]:
# ביטויים לבדיקה
keywords = ["העותק כפוף לשינויי עריכה וניסוח", "העתק מתאים", 'ניתנההיום', 'ניתןהיום', 'ניתן היום', 'ניתנה היום',
            "העתקמתאים", "מזכיר ראשי", "supreme.court.gov.il", "מרכז מידע", "מרכזמידע"]

# בדיקה אילו שורות מכילות אחד מהביטויים
rows_with_keyword = full_rap["גוף המסמך"].apply(
    lambda x: any(keyword in x for keyword in keywords) if isinstance(x, str) else False
)

# סינון שורות שלא מכילות את הביטוי
filtered_data = full_rap[rows_with_keyword]

# הדפסת מספר השורות שנמחקו
num_removed = len(full_rap) - len(filtered_data)  # **שונה לחישוב מדויק**
print(f"\nמספר השורות שנמחקו: {num_removed}")

# **שינוי לצורך סעיף 6**: בדיקה אם נותרו ערכים חסרים בעמודת "גוף המסמך"
missing_values_count = filtered_data["גוף המסמך"].isna().sum()
if missing_values_count > 0:
    print(f"\nנותרו {missing_values_count} ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")
else:
    print("\nאין ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")

# שמירת הנתונים המסוננים לקובץ חדש
filtered_data.to_csv("full_rap_df_cleaned.csv", index=False, encoding='utf-8')
print("\nהנתונים המסוננים נשמרו לקובץ: 'new_filtered_data.csv'")


full_rap = filtered_data.copy()  # **שונה להעתקה בטוחה**


In [ ]:
full_rap

In [ ]:
for col in full_rap.columns:
  print(col)

In [ ]:
sides = full_rap[[
"צד א'",
"צד ב'"
]]

In [ ]:
values = [val for value in sides.values for val in value]
values

In [ ]:
clean_values = set()
for val in values:
    val_str = str(val)  # המרה למחרוזת
    cleaned_val = re.sub(r"[\[\]'\\n]", "", val_str).strip()
    clean_values.add(cleaned_val)

In [ ]:
clean_values

In [ ]:
def check_sub(value,subs):
  for sub in subs:
    if(value.find(sub)!=-1):
      return 1
  return 0

In [ ]:
private_sides = set()
public_sides = set()
for value in clean_values:
  if(check_sub(value,["רשות מקרקעי ישראל","מדינת ","מועצה ","שירות ה","היחידה הארצי","לאכיפת ","הרשות ל","המשרד ל",'יועמ"ש',"המשפטי לממשלה","מנהל מקרקעי","פרקליטות","שירות בתי הסוהר",'שב"ס',"מינהל מקרקעי","עירית ","רשויות","איגוד ערים","עיריית ","מדינת ישראל","משרד ה","רשות ה","אלוף ",'יו"ר',"יושב ראש","מדינת ישראל","ראש עיריית","ועדה","צבא","מפקד","פיקוד","משטרת"])==0):
    private_sides.add(value)
  else:
    public_sides.add(value)

In [ ]:
for side in private_sides:
  print(side)

In [ ]:
for side in public_sides:
  print(side)

In [ ]:
sides_A = full_rap[[
"צד א'"
]]

values_A = [val for value in sides_A.values for val in value]
values_A

In [ ]:
def check_contry_side(value):
    # רשימת מילות מפתח
    subs = [
        "רשות מקרקעי ישראל", "מדינת ", "מועצה ", "שירות ה", "היחידה הארצי", "לאכיפת ", "הרשות ל",
        "המשרד ל", 'יועמ"ש', "המשפטי לממשלה", "מנהל מקרקעי", "פרקליטות", "שירות בתי הסוהר", 'שב"ס',
        "מינהל מקרקעי", "עירית ", "רשויות", "איגוד ערים", "עיריית ", "מדינת ישראל", "משרד ה", "מקרקעי ישראל",
        "רשות ה", "אלוף ", 'יו"ר', "יושב ראש", "ראש עיריית", "ועדה", "צבא", "מפקד", "פיקוד", "משטרת"
    ]

    # ניקוי רווחים והפיכת המחרוזת לאותיות קטנות
    try:
      value = value.strip()

    except AttributeError:
      return 0

    # בדיקה אם אחת המחרוזות קיימת ב-value
    if any(sub in value for sub in subs):
        return 1
    print(value)

    return 0

In [ ]:
check_none = full_rap[full_rap["צד א'"].isna()]
check_none.loc[:,"צד ב'"]

In [ ]:
full_rap[full_rap["צד א'"].isna() & full_rap["צד ב'"].isna()].shape[0]

In [ ]:
full_rap.dropna(subset=["צד א'","צד ב'"],how='all',inplace=True)

In [ ]:
full_rap["מדינת ישראל בצד א'"] = full_rap["צד א'"].apply(lambda x: check_contry_side(x))

# Create checked data - to remove from predictions

In [ ]:
dca = pd.read_csv("dca_for_classifier.csv")

In [ ]:
ignore_df_1 = pd.read_excel("full_data_BOTH_models_with_predictions_V.xlsx")
ignore_df_1.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df_2 = pd.read_excel("2_full_data_with_predictions V2.xlsx")
ignore_df_2.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df_3 = pd.read_excel("full_data_with_predictions-VETTING.xlsx")
ignore_df_3.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df = pd.concat([ignore_df_1,ignore_df_2,ignore_df_3,dca])
ignore_df.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df.drop_duplicates(subset=['מספר הליך'],keep='first',inplace=True)



In [ ]:
ignore_df.shape

In [ ]:
ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()

In [ ]:
ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()].shape

In [ ]:
# ignore_df[(ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'בקשה אחרת') & (ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'דלמטה מיוחד')].shape

In [ ]:
ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()].shape

In [ ]:
delete_from_main_df = ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()]
delete_from_main_df.shape

In [ ]:
ignore_df[(ignore_df['בקשה לרשות ערעור התקבלה?']== 'אחר')].shape

In [ ]:
ignore_df.to_csv("allready_checked_data_with_dca.csv", index=False, encoding='utf-8')

In [ ]:
delete_from_main_df = pd.concat([delete_from_main_df,ignore_df[(ignore_df['בקשה לרשות ערעור התקבלה?'] == 'אחר')]])

In [ ]:
delete_from_main_df.shape

In [ ]:
delete_from_main_df.drop_duplicates(subset=['מספר הליך'],keep='first',inplace=True)

In [ ]:
delete_from_main_df.shape

In [ ]:
full_rap.shape

In [ ]:
full_rap = full_rap[~full_rap["מספר הליך"].isin(delete_from_main_df["מספר הליך"])]

In [ ]:
full_rap.to_csv("full_rap_df_cleaned.csv", index=False, encoding='utf-8')

In [ ]:
full_rap

# Remove checked data from the prediction's sample!!!

In [ ]:
none_checked_raap = full_rap[~full_rap["מספר הליך"].isin(ignore_df["מספר הליך"])]